# Handin Exercise 6

Create a module containing a class with the following methods:
1. __init__(self, url_list)
2. download(url,filename) raises NotFoundException when url returns 404
3. multi_download() uses threads to download multiple urls as text and stores filenames as a property
4. __iter__() returns an iterator
5. __next__() returns the next filename (and stops when there are no more)
6. urllist_generator() returns a generator to loop through the urls
7. avg_vowels(text) - a rough estimate on readability returns average number of vowels in the words of the text
8. hardest_read() returns the filename of the text with the highest vowel score (use all the cpu cores on the computer for this work.

### Ex 2
Create a notebook and import your module from above
1. Find 10 books on https://www.gutenberg.org/browse/scores/top and download them using an object of the class you just created before
2. Test the different methods of your class
3. Make a bar plot with a sorted list of books on x-axis and avg vowels on y-axis

In [40]:
import requests
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import multiprocessing

class FileLoader():
    def __init__(self, url_list=[]):
        self.url_list = url_list
        self.filenames = []
        
    def download(self, url, filename):
        res = requests.get(url)
        
        if(res.ok):
            with open(filename, 'wb') as fd:
                self.filenames.append(filename)
                for chunk in res.iter_content(chunk_size=1024):
                    fd.write(chunk)
        elif res.status == 404:
            raise NotFoundException('Statuscode:', {res.status})

    def multi_download(self, workers=5):
        with ThreadPoolExecutor(workers) as ex:
            for i in range(0, len(self.url_list)):
                print('Book: ', i+1)
                self.download(self.url_list[i], 'book' + str(i+1) + '.txt')
                
    def __iter__(self):
        self.n = 0
        return self
    
    def __next__(self):
        index = self.n
        if index != len(self.filenames):
            self.n += 1
            return self.filenames[index]
        else:
            raise StopIteration
            
    def urllist_generator(self):
        for url in self.url_list:
            yield url
            
class NotFoundException():
    pass

In [41]:
urllist = ['https://www.gutenberg.org/ebooks/64317', 'https://www.gutenberg.org/ebooks/84', 'https://www.gutenberg.org/ebooks/1080',
          'https://www.gutenberg.org/ebooks/174', 'https://www.gutenberg.org/ebooks/5200']
urls = FileLoader(urllist)
FileLoader.multi_download(urls)

Book:  1
Book:  2
Book:  3
Book:  4
Book:  5


In [37]:
[filename for filename in urls]

['book1.txt', 'book2.txt', 'book3.txt', 'book4.txt', 'book5.txt']

In [39]:
FileLoader.urllist_generator(FileLoader)

<generator object FileLoader.urllist_generator at 0x7faf90da6ed0>